In [49]:
import os
import yaml
import pandas as pd

from pathlib import Path
# from sklearn.metrics import ConfusionMatrixDisplay

from ba_dev.utils import load_path_yaml
from ba_dev.eval_helpers import *

set_custom_plot_style()


In [50]:
paths = load_path_yaml('/cfs/earth/scratch/kraftjul/BA/data/path_config.yml')

path_to_models = paths['complete_models']


In [3]:
df = evaluate_all_runs(
    path_to_runs=path_to_models,
    metrics='balanced_accuracy_score',
    )

In [ ]:
# generating the balanced accuracy table

table = (
    df
    .groupby(['model_name', 'pretrained'])
    .agg(
        trainable_params=('trainable_params', 'first'),
        img_score_mean=('img_score', 'mean'),
        img_score_std =('img_score', 'std'),
        seq_score_mean=('seq_score', 'mean'),
        seq_score_std =('seq_score', 'std'),
    )
    .reset_index()
    .sort_values(by='img_score_mean', ascending=False)
    ).rename(columns={
    'model_name': 'Model',
    'pretrained': 'Pretrained',
    'trainable_params': 'Params (M)',
    'img_score_mean': r'$\mu_{\mathrm{img}}$',
    'img_score_std': r'$\sigma_{\mathrm{img}}$',
    'seq_score_mean': r'$\mu_{\mathrm{seq}}$',
    'seq_score_std': r'$\sigma_{\mathrm{seq}}$',
})

table['Pretrained'] = table['Pretrained'].map({True: 'Yes', False: 'No'})
table['Params (M)'] = (table['Params (M)'] / 1_000_000).astype(int)
table['Model'] = table['Model'].str.replace('_', r'\_')

float_fmt = lambda x: f"{x:.4f}"

latex = table.to_latex(
    index=False,
    column_format='l c r c c c c',
    float_format=float_fmt,
    escape=False,
    caption='Balanced accuracy by model and pretraining arranged by image score.',
    label='tab:bal_acc_by_model'
    )

with open(paths['table'] / 'bal_acc_by_model.tex', 'w') as f:
    f.write(latex)